## Test_api

In [1]:
# api_brussels_correct.py
import requests
import json

class BrusselsAPICorrect:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        print("🔄 Génération d'un nouveau token...")
        
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        try:
            response = requests.post(token_url, data=payload)
            print(f"Token Response Status: {response.status_code}")
            
            if response.status_code == 200:
                token_data = response.json()
                new_token = token_data["access_token"]
                expires_in = token_data.get("expires_in", "Inconnu")
                print(f"✅ NOUVEAU TOKEN GÉNÉRÉ!")
                print(f"   Token: {new_token[:20]}...")
                print(f"   Expire dans: {expires_in} secondes")
                return new_token
            else:
                print(f"❌ Erreur génération token: {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Exception token: {e}")
            return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne URL"""
        if not self.access_token:
            print("❌ Aucun token disponible")
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # URL CORRECTE comme dans la documentation
        url = f"{self.base_url}/events"
        params = {"page": page}
        
        print(f"🔍 Appel API: {url}")
        print(f"   Token: {self.access_token[:20]}...")
        
        try:
            response = requests.get(url, headers=headers, params=params)
            print(f"📡 Status Code: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                events = data.get("events", [])
                print(f"✅ SUCCÈS! {len(events)} événements récupérés")
                return events[:limit]  # Limite le nombre
            else:
                print(f"❌ ERREUR API: {response.text}")
                return []
                
        except Exception as e:
            print(f"❌ Exception: {e}")
            return []
    
    def search_events(self, keyword: str):
        """Recherche des événements par mot-clé"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events/search"
        params = {"keyword": keyword}
        
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                return response.json().get("events", [])
            else:
                print(f"Erreur recherche: {response.status_code}")
                return []
        except Exception as e:
            print(f"Exception recherche: {e}")
            return []

# Test complet
if __name__ == "__main__":
    print("🧪 API BRUSSELS AGENDA - VERSION CORRIGÉE")
    print("=" * 60)
    
    api = BrusselsAPICorrect()
    
    if api.access_token:
        print("\n📅 RÉCUPÉRATION DES ÉVÉNEMENTS...")
        events = api.get_events(page=1, limit=5)
        
        if events:
            print(f"\n🎉 {len(events)} ÉVÉNEMENTS TROUVÉS:")
            print("-" * 50)
            
            for i, event in enumerate(events, 1):
                print(f"\n{i}. {event.get('title', 'Sans titre')}")
                print(f"   📍 {event.get('location', {}).get('name', 'Lieu non spécifié')}")
                print(f"   📅 {event.get('date', {}).get('start', 'Date non spécifiée')}")
                print(f"   🔗 {event.get('url', 'Pas de lien')}")
                
                # Affiche un extrait de la description
                description = event.get('description', '')
                if description:
                    print(f"   📝 {description[:100]}...")
        else:
            print("😞 Aucun événement trouvé")
            
        # Test de recherche
        print("\n🔍 TEST DE RECHERCHE 'concert'...")
        concerts = api.search_events("concert")
        print(f"🎵 {len(concerts)} concerts trouvés")
        
    else:
        print("❌ Impossible d'initialiser l'API")

🧪 API BRUSSELS AGENDA - VERSION CORRIGÉE
🔄 Génération d'un nouveau token...
Token Response Status: 200
✅ NOUVEAU TOKEN GÉNÉRÉ!
   Token: 097590bb-eca0-35c4-9...
   Expire dans: 9223372036854775 secondes

📅 RÉCUPÉRATION DES ÉVÉNEMENTS...
🔍 Appel API: https://api.brussels:443/api/agenda/0.0.1/events
   Token: 097590bb-eca0-35c4-9...
📡 Status Code: 200
✅ SUCCÈS! 0 événements récupérés
😞 Aucun événement trouvé

🔍 TEST DE RECHERCHE 'concert'...
Erreur recherche: 500
🎵 0 concerts trouvés


In [15]:
# api_brussels_fixed_v2.py
import requests
import json
from datetime import datetime

class BrusselsAPIWorking:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        # self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne structure"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # url = f"{self.base_url}/events"
        url = f"{self.base_url}/events/category"
        # params = {"page": page}
        params = {"mainCategory": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # EXTRACTION CORRECTE des événements
            events = data.get("response", {}).get("results", {}).get("event", [])
            
            print(f"📊 Statistiques:")
            print(f"   - Page: {data.get('response', {}).get('page', 1)}")
            print(f"   - Total pages: {data.get('response', {}).get('pageCount', 1)}")
            print(f"   - Total événements: {data.get('response', {}).get('resultCount', 0)}")
            print(f"   - Événements cette page: {len(events)}")
            
            return events[:limit]
        else:
            print(f"❌ Erreur API: {response.status_code}")
            return []
    
    def debug_event_structure(self, event):
        """Debug la structure d'un événement"""
        print("\n🔍 STRUCTURE DE L'ÉVÉNEMENT:")
        print(json.dumps(event, indent=2, ensure_ascii=False)[:1000] + "...")
    
    def format_event(self, event):
        """Formate un événement pour l'affichage"""
        # Debug si nécessaire
        # self.debug_event_structure(event)
        
        # Gère les traductions
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        nl_translation = translations.get("nl", {})
        
        title = fr_translation.get("title") or nl_translation.get("title") or "Sans titre"
        description = fr_translation.get("shortdescr") or nl_translation.get("shortdescr") or ""
        
        # DATE - La structure est différente
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        end_date = ""
        
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", "Date non spécifiée")
                end_date = first_date.get("end", "")
        
        # Lieu
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        # Prix
        price_info = event.get("price", {})
        if isinstance(price_info, dict):
            price = price_info.get("base", "Gratuit" if event.get("is_free", False) else "Payant")
        else:
            price = "Gratuit" if event.get("is_free", False) else "Payant"
        
        return {
            "title": title,
            "description": description[:200] + "..." if len(description) > 200 else description,
            "start_date": start_date,
            "end_date": end_date,
            "location": place_name,
            "url": fr_translation.get("website") or nl_translation.get("website"),
            "price": price,
            "is_free": event.get("is_free", False)
        }
    
    def get_social_events(self, limit=6):
        """Récupère des événements sociaux spécifiques"""
        events = self.get_events(limit=limit)
        social_events = []
        
        for event in events:
            formatted = self.format_event(event)
            
            # Filtre pour événements sociaux (adaptable)
            if any(keyword in formatted['title'].lower() for keyword in 
                  ['atelier', 'workshop', 'rencontre', 'meeting', 'café', 'débat', 'discussion']):
                social_events.append(formatted)
            else:
                # Inclut tous les événements pour le POC
                social_events.append(formatted)
        
        return social_events

# Test des événements
if __name__ == "__main__":
    print("🎉 API BRUSSELS - VERSION CORRIGÉE!")
    print("=" * 60)
    
    api = BrusselsAPIWorking()
    
    if api.access_token:
        print("✅ Token valide")
        
        # Récupère les événements
        events = api.get_events(page=1, limit=6)
        
        if events:
            print(f"\n📅 AFFICHAGE DE {len(events)} ÉVÉNEMENTS:")
            print("=" * 60)
            
            for i, event in enumerate(events, 1):
                try:
                    formatted_event = api.format_event(event)
                    print(f"\n{i}. {formatted_event['title']}")
                    print(f"   📍 {formatted_event['location']}")
                    print(f"   📅 {formatted_event['start_date']}")
                    if formatted_event['end_date']:
                        print(f"   ➡️  {formatted_event['end_date']}")
                    print(f"   💰 {formatted_event['price']}")
                    if formatted_event['description']:
                        print(f"   📝 {formatted_event['description']}")
                    if formatted_event['url']:
                        print(f"   🔗 {formatted_event['url'][:50]}...")
                    
                except Exception as e:
                    print(f"\n{i}. ❌ Erreur formatage: {e}")
                    # Debug cet événement problématique
                    api.debug_event_structure(event)
                
                if i % 2 == 0:  # Séparateur tous les 2 événements
                    print("-" * 50)
                    
        else:
            print("😞 Aucun événement trouvé")
            
    else:
        print("❌ Token invalide")

🎉 API BRUSSELS - VERSION CORRIGÉE!
✅ Token valide
📊 Statistiques:
   - Page: 1
   - Total pages: 41
   - Total événements: 1009
   - Événements cette page: 25

📅 AFFICHAGE DE 6 ÉVÉNEMENTS:

1. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   🔗 https://www.art-base.be...

2. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   📝 La sagesse des petits riens : Un voyage sensible dans l'art de la présence, de la lenteur et des détails, accompagné d'improvisations poétiques au piano.
   🔗 https://www.jardinmusical.org/infos-evenement/la-s...
--------------------------------------------------

3. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée
   💰 Payant
   📝 C'est bientôt, c'est Mardi ! 
Le Effel Quartet, quatuor de saxophones, nous accueille avec un verre offert à partir de 18h30 et nous convie au concert à 19h00.


   🔗 https://chambermusiceurope.org/classique-ici-effel...

4. Sans titre
   📍 Lieu non spécifié
   📅 Date non spécifiée


In [7]:
# api_brussels_filtered.py
import requests
import json
from datetime import datetime, timedelta
import re

class BrusselsAPIFiltered:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels/api/agenda/0.0.1/events/category?mainCategory=1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, page=1, limit=20):
        """Récupère les événements avec la bonne structure"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": page}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            
            print(f"📊 Récupération: {len(events)} événements bruts")
            return events
        else:
            print(f"❌ Erreur API: {response.status_code}")
            return []
    
    def is_french_event(self, event):
        """Vérifie si l'événement a du contenu en français"""
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        
        # Vérifie qu'il y a au moins un titre en français
        return bool(fr_translation.get("title"))
    
    def is_future_event(self, event):
        """Vérifie si l'événement est aujourd'hui ou dans le futur"""
        dates = event.get("dates", [])
        if not dates or not isinstance(dates, list):
            return False
        
        for date_range in dates:
            if isinstance(date_range, dict):
                end_date_str = date_range.get("end")
                if end_date_str:
                    try:
                        # Convertit la date de fin en objet datetime
                        end_date = datetime.fromisoformat(end_date_str.replace('Z', '+00:00'))
                        # Garde si finit aujourd'hui ou dans le futur
                        return end_date.date() >= datetime.now().date()
                    except:
                        continue
        return False
    
    def is_in_brussels_area(self, event):
        """Vérifie si l'événement est dans la région Bruxelles"""
        place = event.get("place", {})
        if isinstance(place, dict):
            place_name = place.get("name", "").lower()
            # Liste des communes bruxelloises et environs proches
            brussels_areas = [
                'bruxelles', 'brussel', 'ixelles', 'etterbeek', 'woluwe', 'uccle', 
                'anderlecht', 'molenbeek', 'schaerbeek', 'saint-gilles', 'forest',
                'berchem', 'ganshoren', 'jette', 'koekelberg', 'evere', 'watermael',
                'woluwe-saint-pierre', 'woluwe-saint-lambert', 'saint-josse'
            ]
            return any(area in place_name for area in brussels_areas)
        return False
    
    def format_event_date(self, date_str):
        """Formate la date en français"""
        try:
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            return date_obj.strftime("%d/%m/%Y à %H:%M")
        except:
            return date_str
    
    def format_event(self, event):
        """Formate un événement pour l'affichage"""
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        
        title = fr_translation.get("title", "Sans titre")
        description = fr_translation.get("shortdescr", "") or fr_translation.get("longdescr", "")
        
        # Dates
        dates = event.get("dates", [])
        date_info = "Date non spécifiée"
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_str = first_date.get("start")
                end_str = first_date.get("end")
                if start_str and end_str:
                    date_info = f"{self.format_event_date(start_str)} → {self.format_event_date(end_str)}"
                elif start_str:
                    date_info = self.format_event_date(start_str)
        
        # Lieu
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else "Lieu non spécifié"
        
        # Prix
        price = "Gratuit" if event.get("is_free", False) else "Payant"
        price_info = event.get("price", {})
        if isinstance(price_info, dict) and price_info.get("base"):
            price = f"{price_info['base']} €"
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "date": date_info,
            "location": place_name,
            "url": fr_translation.get("website", ""),
            "price": price,
            "is_free": event.get("is_free", False),
            "raw_start_date": dates[0].get("start") if dates and isinstance(dates, list) and dates else ""
        }
    
    def get_social_events_filtered(self, limit=8):
        """Récupère des événements sociaux filtrés"""
        print("🔄 Récupération et filtrage des événements...")
        
        all_events = self.get_events(limit=50)  # Récupère plus pour filtrer
        filtered_events = []
        
        for event in all_events:
            # Applique les filtres
            if (self.is_french_event(event) and 
                self.is_future_event(event) and 
                self.is_in_brussels_area(event)):
                
                formatted = self.format_event(event)
                filtered_events.append((event, formatted))
        
        # Trie par date de début
        filtered_events.sort(key=lambda x: x[1]["raw_start_date"] or "")
        
        print(f"✅ {len(filtered_events)} événements après filtrage")
        return [formatted for _, formatted in filtered_events[:limit]]
    
    def search_social_keywords(self, events):
        """Filtre les événements avec mots-clés sociaux"""
        social_keywords = [
            'atelier', 'workshop', 'rencontre', 'meeting', 'café', 'débat', 
            'discussion', 'conférence', 'formation', 'cours', 'gratuit',
            'communauté', 'social', 'échange', 'partage', 'collectif'
        ]
        
        social_events = []
        for event in events:
            title_lower = event["title"].lower()
            desc_lower = event["description"].lower()
            
            if any(keyword in title_lower or keyword in desc_lower for keyword in social_keywords):
                social_events.append(event)
        
        return social_events

# Test des événements filtrés
if __name__ == "__main__":
    print("🎉 API BRUSSELS - ÉVÉNEMENTS FILTRÉS!")
    print("=" * 60)
    
    api = BrusselsAPIFiltered()
    
    if api.access_token:
        print("✅ Token valide")
        
        # Récupère les événements filtrés
        events = api.get_social_events_filtered(limit=8)
        
        if events:
            print(f"\n📅 {len(events)} ÉVÉNEMENTS FRANÇAIS À BRUXELLES (AUJOURD'HUI/FUTUR):")
            print("=" * 60)
            
            for i, event in enumerate(events, 1):
                print(f"\n{i}. 🎯 {event['title']}")
                print(f"   📍 {event['location']}")
                print(f"   📅 {event['date']}")
                print(f"   💰 {event['price']}")
                if event['description']:
                    print(f"   📝 {event['description']}")
                if event['url']:
                    print(f"   🔗 {event['url'][:60]}...")
                
                if i % 2 == 0:
                    print("-" * 50)
            
            # Événements spéciaux sociaux
            print(f"\n🤝 ÉVÉNEMENTS SPÉCIALEMENT SOCIAUX:")
            social_events = api.search_social_keywords(events)
            for i, event in enumerate(social_events[:3], 1):
                print(f"   {i}. {event['title']}")
                    
        else:
            print("😞 Aucun événement trouvé avec les filtres actuels")
            
    else:
        print("❌ Token invalide")

🎉 API BRUSSELS - ÉVÉNEMENTS FILTRÉS!
✅ Token valide
🔄 Récupération et filtrage des événements...
❌ Erreur API: 500
✅ 0 événements après filtrage
😞 Aucun événement trouvé avec les filtres actuels


## Test petit agent

In [17]:
!pip install langchain langchain-mistralai python-dotenv requests

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/8e/4a/02c14af46fa79ce7b02a0f8af46f5905cc7e8b647a5f1a7c793c03ac5063/langchain-1.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<2.0.0,>=1.0.4 from https://files.pythonhosted.org/packages/6e/ee/aaf2343a35080154c82ceb110e03dd00f15459bc72e518df51724cbc41a9/langchain_core-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for langgraph<1.1.0,>=1.0.2 from https://files.pythonhosted.org/packages/84/a3/fdf6ecd0e44cb02d20afe7d0fb64c748a749f4b2e011bf9a785a32642367/langgraph-1.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for langgraph-checkpoint<4.0.0,>=2.1.0 from https://files.pythonhosted.org/packages/48/e3/616e3a7ff737d98c1bbb5700dd62278914e2a9ded09a79a1fa93cf24ce12/langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for langgraph-prebuilt<1.1.0,>=1.0.2 from https://files.pythonhosted.org/packag

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.12.0 requires huggingface-hub<1.0.0,>=0.11.0, but you have huggingface-hub 1.0.1 which is incompatible.


In [18]:
# social_agent.py
from langchain.agents import AgentType, initialize_agent
from langchain.mistralai import ChatMistralAI
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, category=None, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        if category:
            url = f"{self.base_url}/events/category"
            params = {"mainCategory": category}
        else:
            url = f"{self.base_url}/events"
            params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        nl_translation = translations.get("nl", {})
        
        title = fr_translation.get("title") or nl_translation.get("title") or "Sans titre"
        description = fr_translation.get("shortdescr") or nl_translation.get("shortdescr") or ""
        
        # Date
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        # Lieu
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False),
            "url": fr_translation.get("website") or nl_translation.get("website")
        }

class SocialConnectionAgent:
    def __init__(self):
        # LLM Mistral
        self.llm = ChatMistralAI(
            model="mistral-large-latest",
            temperature=0.3,
            mistral_api_key=os.getenv("MISTRAL_API_KEY")
        )
        
        # API Brussels
        self.api = BrusselsAPI()
        
        # Mémoire de conversation
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Tools LangChain
        self.tools = self._setup_tools()
        
        # Agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            agent_kwargs={
                "system_message": SystemMessage(content=self._get_system_prompt()),
                "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")]
            }
        )
    
    def _get_system_prompt(self):
        return """Tu es un assistant bienveillant qui aide les personnes isolées à retrouver du lien social à Bruxelles.

Ton rôle:
- Proposer des activités sociales adaptées au profil et aux intérêts
- Être encourageant et bienveillant
- Suggérer des activités progressives (d'abord en petit groupe, puis plus grandes)
- Tenir compte des contraintes (budget, mobilité, etc.)
- Toujours proposer des activités concrètes avec dates et lieux

Ton ton est chaleureux, empathique et motivant."""
    
    def _setup_tools(self):
        def find_social_activities(query: str) -> str:
            """Trouve des activités sociales à Bruxelles basées sur les intérêts"""
            # Détecte le type d'activité demandé
            query_lower = query.lower()
            
            # Mapping des catégories (à adapter selon l'API)
            if any(word in query_lower for word in ['concert', 'musique', 'music']):
                events = self.api.get_events(category=1, limit=5)  # Musique
            elif any(word in query_lower for word in ['atelier', 'workshop', 'cours', 'apprendre']):
                events = self.api.get_events(category=2, limit=5)  # Ateliers
            elif any(word in query_lower for word in ['sport', 'yoga', 'danse']):
                events = self.api.get_events(category=3, limit=5)  # Sports
            else:
                # Activités sociales générales
                events = self.api.get_events(limit=6)
            
            if not events:
                return "Je n'ai pas trouvé d'activités sociales pour le moment. Essayez de reformuler votre recherche."
            
            # Formate les événements
            result = "Voici des activités sociales à Bruxelles qui pourraient vous intéresser:\n\n"
            
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event(event)
                result += f"🎯 {formatted['title']}\n"
                result += f"   📍 Où: {formatted['location']}\n"
                result += f"   📅 Quand: {formatted['start_date']}\n"
                result += f"   💰 Prix: {'Gratuit' if formatted['is_free'] else 'Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            result += "\n💡 Conseil: Ces activités sont d'excellentes occasions de rencontrer de nouvelles personnes !"
            return result
        
        def find_free_activities(query: str) -> str:
            """Trouve des activités gratuites pour socialiser"""
            events = self.api.get_events(limit=8)
            free_events = [e for e in events if e.get('is_free', False)]
            
            if not free_events:
                return "Je n'ai pas trouvé d'activités gratuites pour le moment."
            
            result = "Activités gratuites pour socialiser sans dépenser:\n\n"
            for i, event in enumerate(free_events[:4], 1):
                formatted = self.api.format_event(event)
                result += f"🎯 {formatted['title']}\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n\n"
            
            return result
        
        return [
            Tool(
                name="RechercheActivitesSociales",
                func=find_social_activities,
                description="Recherche des activités sociales et culturelles à Bruxelles. Utilise quand quelqu'un cherche à rencontrer des gens ou à socialiser."
            ),
            Tool(
                name="ActivitesGratuites", 
                func=find_free_activities,
                description="Trouve des activités gratuites pour socialiser sans budget. Parfait pour les personnes avec des contraintes financières."
            )
        ]
    
    def chat(self, message: str) -> str:
        """Interagit avec l'agent"""
        try:
            response = self.agent.run(message)
            return response
        except Exception as e:
            return f"Je rencontre un problème technique. Pouvez-vous reformuler votre demande? Erreur: {str(e)}"
        
    def interactive_chat():
        """Mode conversation interactif"""
        print("🤖 AGENT DE CONNEXION SOCIALE - MODE INTERACTIF")
        print("=" * 60)
        print("Tapez 'quit' pour arrêter la conversation\n")
        
        agent = SocialConnectionAgent()
        
        while True:
            user_input = input("👤 Vous: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("🤖 Agent: Au revoir ! Bonne chance pour vos rencontres sociales 🌟")
                break
                
            if user_input:
                print("🤖 Agent: ", end="")
                response = agent.chat(user_input)
                print(response)
                print()

# Test de l'agent
if __name__ == "__main__":
    print("Choisissez le mode:")
    print("1 - Mode test automatique")
    print("2 - Mode conversation interactif")
    
    choice = input("Votre choix (1 ou 2): ").strip()
    
    if choice == "2":
        interactive_chat()
    else:
        # Mode test automatique original
        print("🤖 AGENT DE CONNEXION SOCIALE")
        print("=" * 50)
        print("Agent: Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles.")
        print("        Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.\n")
        
        agent = SocialConnectionAgent()
        
        test_scenarios = [
            "Bonjour, je me sens un peu seul et j'aimerais rencontrer des gens",
            "Je cherche des activités gratuites pour socialiser", 
            "Quels concerts y a-t-il en ce moment?",
            "Je suis nouveau à Bruxelles, que me conseilles-tu?"
        ]
        
        for scenario in test_scenarios:
            print(f"👤: {scenario}")
            response = agent.chat(scenario)
            print(f"🤖: {response}")
            print("-" * 80)

ImportError: cannot import name 'AgentType' from 'langchain.agents' (c:\Users\ziani\anaconda3\Lib\site-packages\langchain\agents\__init__.py)